In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

In [ ]:
p = 7
n = 10000
inputs = []
outputs = []

def f(x,y):
  return (x-y)%p




for i in range(n):
  x = np.random.randint(0, p)
  y = np.random.randint(0, p)
  inputs.append([x,y])
  outputs.append([f(x,y)])

In [ ]:
p = 7
n = 10000
inputs = []
outputs = []

def g(seq):
  if len(seq)<5:
    seq += [0]*(5-len(seq))
  return seq[0]-seq[1]+seq[2]-seq[3]+seq[4]
for i in range(n):
  l = np.random.randint(0,6)
  temp = [np.random.randint(0, p) for j in range(l)]
  inputs.append(temp)
  outputs.append(g(temp))

# print(inputs)
# print(outputs)

# print(g([0,2,0]))

In [ ]:
X = np.reshape(inputs, (len(inputs), len(inputs[0]), 1))
X = X/p # normalize
y = np_utils.to_categorical(outputs)

model = Sequential()
# Note: all LSTM layers except the last must have 'return_sequences=True'
model.add(LSTM(128, input_shape=(X.shape[1], 1), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(8))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

history = model.fit(X, y, epochs=50, batch_size=64,\
                    verbose=2, validation_split=0.2)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_65 (LSTM)               (None, 5, 128)            66560     
_________________________________________________________________
lstm_66 (LSTM)               (None, 5, 64)             49408     
_________________________________________________________________
lstm_67 (LSTM)               (None, 5, 32)             12416     
_________________________________________________________________
dropout_35 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
lstm_68 (LSTM)               (None, 5, 16)             3136      
_________________________________________________________________
dropout_36 (Dropout)         (None, 5, 16)             0         
_________________________________________________________________
lstm_69 (LSTM)               (None, 5, 16)           

In [ ]:
# text prediction

from urllib.request import urlopen
import re

url = "https://www.gutenberg.org/files/1661/1661-0.txt"
file = urlopen(url)
text = file.read().decode("utf-8").lower()

# preprocess the text to remove punctuation, non-alphabetic char, multiple spaces
text = re.sub('[^a-zA-Z]', ' ', text)
text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
text = re.sub(r'\s+', ' ', text)



print('# of total char: ', len(text))
unique_char = sorted(list(set(text)))
print('# of unique char: ', len(unique_char))

#to map unique characters to integers
i2c = {}
c2i = {}
for idx,char in enumerate(unique_char):
  i2c[idx] = char
  c2i[char] = idx

#define input and output dataset by chunking our text

#each subsentence has k characters
k = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - k, step):
    sentences.append(text[i: i + k])
    next_chars.append(text[i + k])
#convert the information to bool represented array

x = np.zeros((len(sentences), k, len(unique_char)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_char)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for c, char in enumerate(sentence):
        x[i, c, c2i[char]] = 1
    y[i, c2i[next_chars[i]]] = 1

# print(x)
# print(y)


# of total char:  543516
# of unique char:  27


In [ ]:
# print(i2c)
# print(x[10][5])
# print(y[10])

[False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False]
[False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False]


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(k, len(unique_char)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['acc'])

#This part is not original!! Reference to https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
# define the checkpoint
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
history = model.fit(x, y, epochs=10, batch_size=128, callbacks=callbacks_list,validation_split=0.2)

# history = model.fit(x, y, batch_size=128, epochs=5,validation_split=0.2)

Epoch 1/10
1133/1133 [==============================] - 398s 348ms/step - loss: 2.5939 - acc: 0.2525 - val_loss: 2.0386 - val_acc: 0.3955

Epoch 00001: loss improved from inf to 2.33042, saving model to weights-improvement-01-2.3304.hdf5
Epoch 2/10
1133/1133 [==============================] - 393s 347ms/step - loss: 1.9792 - acc: 0.4074 - val_loss: 1.8402 - val_acc: 0.4412

Epoch 00002: loss improved from 2.33042 to 1.93230, saving model to weights-improvement-02-1.9323.hdf5
Epoch 3/10
1133/1133 [==============================] - 392s 346ms/step - loss: 1.8151 - acc: 0.4475 - val_loss: 1.7408 - val_acc: 0.4709

Epoch 00003: loss improved from 1.93230 to 1.79146, saving model to weights-improvement-03-1.7915.hdf5
Epoch 4/10
1133/1133 [==============================] - 392s 346ms/step - loss: 1.7237 - acc: 0.4703 - val_loss: 1.6811 - val_acc: 0.4862

Epoch 00004: loss improved from 1.79146 to 1.70676, saving model to weights-improvement-04-1.7068.hdf5
Epoch 5/10
1133/1133 [==============

In [ ]:
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                23552     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 27)                1755      
Total params: 25,307
Trainable params: 25,307
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import random
def random_generate(length):
  index_list = [i for i in range(0,27)]
  #pick a random index to start from our sentences list
  start = np.random.randint(0, len(sentences)-1)
  current_sen = sentences[start]
  current = np.reshape(x[start],(1,50,27))
  # print(current)
  print(current_sen)
  #pict the next char from the predicted distribution
  for i in range(length):
    # print(model.predict(current))
    #choose according to the distribution
    next_idx = random.choices(index_list, weights=model.predict(current)[0], k=1)[0]
    
    
    next_char = i2c[next_idx]
    print(next_char,end='')
    current_sen += next_char
    current_sen = current_sen[1:]

    # convert sentence to our input by changing the last entry of current
    temp = np.zeros((1,27), dtype=np.bool)
    temp[0,c2i[next_char]] = 1
    # print(temp)
    # print(current.shape)
    temp1 = np.append(current[0],temp,axis=0)
    current[0] = np.delete(temp1,1,axis=0)

#generate a sentence with 200 characters
random_generate(200)

hen much surprised and interested on glancing down
 to care to keott mattersial and the craspired she of the some it alarr that the brights he seemed me had beated throught out your fortise car you is all fact of brive as instisting time when she conf

In [ ]:
#to save the model
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Save the weights and model
model.save_weights('./checkpoints/problem3')
model.save('saved_model/problem3') 

INFO:tensorflow:Assets written to: saved_model/problem3/assets


INFO:tensorflow:Assets written to: saved_model/problem3/assets
